In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf

In [ ]:
def get_data(stocks, start, end):
  stockData = yf.download(stocks, start=start, end=end)['Close']
  returns = stockData.pct_change()
  meanReturns = returns.mean()
  covMatrix = returns.cov()
  return meanReturns, covMatrix

In [ ]:
def cagr(initial, final, years):
  return ((final / initial) ** (1 / years) - 1)*100

In [ ]:
stocklist = ['AMZN', 'BABA', 'EBAY', 'SHOP', 'RKUNF', 'PDD', 'MELI']
stocks = [stock for stock in stocklist]
startDate = '2019-05-01'
endDate = '2022-05-01'
meanReturns, covMatrix = get_data(stocks, startDate, endDate)

weights = np.random.random(len(meanReturns))
weights /= np.sum(weights)

mc_sims = 100
T=100

meanM = np.full(shape=(T,len(weights)),fill_value=meanReturns)
meanM = meanM.T

portfolio_sims = np.full(shape=(T,mc_sims),fill_value=0.0)

initialPortfolio = 10000

In [ ]:
for m in range(0, mc_sims):
  Z = np.random.normal(size=(T ,len(weights)))
  L = np.linalg.cholesky(covMatrix)
  dailyReturns = meanM + np.inner(L,Z)
  portfolio_sims[:,m] = np.cumprod(np.inner(weights,dailyReturns.T)+1)*initialPortfolio

final_portfolio_value = portfolio_sims[-1,:]

maxfinalportfolio = max(final_portfolio_value)

In [ ]:
#investment time calc
startDateObj = dt.datetime.strptime(startDate, '%Y-%m-%d')
endDateObj = dt.datetime.strptime(endDate, '%Y-%m-%d')
date_diff = endDateObj - startDateObj
years_diff = date_diff.days / 365

cagr_list = []

In [ ]:
for f in final_portfolio_value:
  cagr_value = cagr(initialPortfolio, f, years_diff)
  cagr_list.append(cagr_value)

average_cagr = sum(cagr_list) / len(cagr_list)
print("The average cagr comes out to be : ",average_cagr,"%")

max_cagr = cagr(initialPortfolio,maxfinalportfolio,years_diff)

print("The maximum cagr comes out to be : ",max_cagr,"%")

In [ ]:
plt.plot(portfolio_sims)
plt.ylabel('portfolio Value ($)')
plt.xlabel('Days')
plt.title('MC simulation of a stock portfolio')
plt.show()